In [1]:
import os
import sys
import requests
from pathlib import Path
import numpy as np
import plotly.express as px
from tqdm import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange, reduce, repeat
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
from fancy_einsum import einsum
import math 

device = t.device("cuda" if t.cuda.is_available() else "cpu")

Create the toy dataset

There are two options for input sequences: noise and signal. The signal contains a 0 preceded by 2 numbers less in [1,4], and has a ground truth answer given by an arbitrary map. 

In [2]:
ans_map = {(1, 1):0, (1, 2):1, (1, 3):2, (1, 4):3, (2, 2):4, (2, 3):5, (2, 4):6, (3,3):7, (3, 4):8, (4, 4):9}
class SkipTrigramDataset(Dataset):

    def __init__(self, size: int, prob:float, prompt_length: int, seed:int = 42):
        self.size = size
        self.prob = prob
        self.prompt_length = prompt_length
        self.data = [self.generate_sample() for _ in range(size)]
        t.manual_seed(seed=seed)

    def generate_sample(self):
        random_tensor = t.randint(low = 5, high = 11, size = (self.prompt_length,))
        random_int = t.randint(low = 0, high = self.prob, size = (1,))
        if random_int.item() == self.prob - 1:
            rand_num_1 = t.randint(1, 5, size = (1,)).item()
            rand_num_2 = t.randint(1, 5, size = (1,)).item()
            random_tensor[0] = rand_num_1
            random_tensor[-2] = rand_num_2
            sample = random_tensor
            sample[-1] = 0
            label = ans_map[tuple(sorted((rand_num_1, rand_num_2)))]
            label = t.tensor([label])
        else:
            sample = random_tensor
            label = t.randint(low = 0, high = 11, size = (1,))

        return sample, label


    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.size

In [3]:
ans_map = {(1, 1):0, (1, 2):1, (1, 3):2, (1, 4):3, (2, 2):4, (2, 3):5, (2, 4):6, (3,3):7, (3, 4):8, (4, 4):9}

example_dataset = SkipTrigramDataset(size = 10_000, prob = 2, prompt_length = 4)
for i in range(10):
    print(f'Sample: {example_dataset[i][0]}, label: {example_dataset[i][1][-1]}')



Sample: tensor([4, 9, 1, 0]), label: 3
Sample: tensor([8, 6, 9, 5]), label: 8
Sample: tensor([ 8,  9,  8, 10]), label: 7
Sample: tensor([10,  7,  6,  5]), label: 6
Sample: tensor([ 9,  5, 10,  6]), label: 1
Sample: tensor([2, 9, 2, 0]), label: 4
Sample: tensor([ 6,  5,  7, 10]), label: 5
Sample: tensor([ 5, 10,  7, 10]), label: 9
Sample: tensor([5, 7, 6, 9]), label: 4
Sample: tensor([3, 7, 3, 0]), label: 7


In [4]:
signal = [example[1].item() for example in example_dataset if example[0][-1] == 0]
noise = [example[1].item() for example in example_dataset if example[0][-1] != 0]

print(len(signal), len(noise))

px.histogram(signal)
#px.histogram(noise)

## repeated ints < 5 are half as likely in the signal dataset. This shouldn't matter, but could check to see if those outputs (0, 4, 7, 9) are treated systematically differently by the model


5061 4939


In [5]:
@dataclass
class model_cfg:
    d_model: int = 12
    d_vocab: int = 12
    init_range: float = 0.02
    n_ctx: int = 5
    d_head: int = 3
    n_heads: int = 3
    n_layers: int = 1

model_config = model_cfg()

print(model_config)

model_cfg(d_model=12, d_vocab=12, init_range=0.02, n_ctx=5, d_head=3, n_heads=3, n_layers=1)


In [6]:
class OneHotEmbed(nn.Module):
    def __init__(self, model_cfg):
        super().__init__()
        self.model_cfg = model_cfg
        self.embedding = nn.Embedding(self.model_cfg.d_vocab, self.model_cfg.d_vocab)
        self.embedding.weight = nn.Parameter(t.eye(self.model_cfg.d_vocab))
        self.embedding.weight.requires_grad = False

    def forward(self, input):
        # tokens: [batch, position]
        # output: [batch, position, d_model]
        input = input.to(device)
        embed_out = self.embedding(input)
        return embed_out
    
class OneHotUnembed(nn.Module):
    def __init__(self, model_cfg):
        super().__init__()
        self.model_cfg = model_cfg
        self.unembed = t.eye(self.model_cfg.d_vocab)

    def forward(self, input):
        # tokens: [batch, position, d_model]
        # output: [batch, position]
        input = input.to(device)
        return t.einsum("vm,bpm->bpv", self.unembed.to(device), input)  # [batch, pos, d_vocab]


In [7]:
class Attention(nn.Module):
    def __init__(self, model_cfg):
        super().__init__()
        self.model_cfg = model_cfg

        # set weights, biases as parameters 
        self.W_Q = nn.Parameter(t.empty((model_cfg.n_heads, model_cfg.d_model, model_cfg.d_head)))
        self.W_K = nn.Parameter(t.empty((model_cfg.n_heads, model_cfg.d_model, model_cfg.d_head)))
        self.W_V = nn.Parameter(t.empty((model_cfg.n_heads, model_cfg.d_model, model_cfg.d_head)))
        self.W_O = nn.Parameter(t.empty((model_cfg.n_heads, model_cfg.d_head, model_cfg.d_model)))
        self.b_Q = nn.Parameter(t.zeros((model_cfg.n_heads, model_cfg.d_head)))
        self.b_K = nn.Parameter(t.zeros((model_cfg.n_heads, model_cfg.d_head)))
        self.b_V = nn.Parameter(t.zeros((model_cfg.n_heads, model_cfg.d_head)))
        self.b_O = nn.Parameter(t.zeros((model_cfg.d_model)))

        #initialize randomly
        nn.init.normal_(self.W_Q, std=self.model_cfg.init_range)
        nn.init.normal_(self.W_K, std=self.model_cfg.init_range)
        nn.init.normal_(self.W_V, std=self.model_cfg.init_range)
        nn.init.normal_(self.W_O, std=self.model_cfg.init_range)
        self.register_buffer("IGNORE", t.tensor(-1e5, dtype=t.float32, device=device))

        self.attn_activation_cache = {}

    def forward(self, normalized_resid_pre: t.Tensor):
        # normalized_resid_pre: [batch, position, d_model] (we're not using LN, so just input)
        # output: [batch, position, d_model]
        normalized_resid_pre = normalized_resid_pre.to(device)

        queries = einsum('n_heads d_model d_head, batch pos_q d_model -> batch pos_q n_heads d_head', self.W_Q, normalized_resid_pre) + self.b_Q
        keys = einsum('n_heads d_model d_head, batch pos_k d_model -> batch pos_k n_heads d_head', self.W_K, normalized_resid_pre) + self.b_K
        values = einsum('n_heads d_model d_head, batch pos_k d_model -> batch pos_k n_heads d_head', self.W_V, normalized_resid_pre) + self.b_V

        attn_scores = einsum('batch pos_q n_heads d_head, batch pos_k n_heads d_head -> batch n_heads pos_q pos_k', queries, keys)

        scaled_attn = attn_scores / math.sqrt(self.model_cfg.d_head)
        masked_attn = self.apply_causal_mask(scaled_attn)
        # apply softmax over keys to get attention pattern
        softmax_attn = masked_attn.softmax(dim = -1)

        # get weighted attention values
        z = einsum('batch pos_k n_heads d_head, batch n_heads pos_q pos_k -> batch pos_q n_heads d_head', values, softmax_attn)

        # layer output
        output = einsum('n_heads d_head d_model, batch pos_q n_heads d_head -> batch pos_q d_model', self.W_O, z) + self.b_O

        ## save to the cache 
        self.attn_activation_cache['hook_q'] = queries
        self.attn_activation_cache['hook_k'] = keys
        self.attn_activation_cache['hook_attn_scores'] = attn_scores
        self.attn_activation_cache['hook_pattern'] = softmax_attn
        self.attn_activation_cache['hook_v'] = values
        self.attn_activation_cache['hook_z'] = z
        self.attn_activation_cache['hook_attn_out'] = output

        return output

    def apply_causal_mask(self, attn_scores: t.Tensor):
        # attn_scores: [batch, n_heads, query_pos, key_pos]
        # output: [batch, n_heads, query_pos, key_pos]
        mask = t.triu(attn_scores, diagonal=1).bool()
        return attn_scores.masked_fill_(mask, self.IGNORE)

define a one layer attn only class      

In [8]:
class OneLayerAttnOnly(nn.Module):
    def __init__(self, model_cfg):
        super().__init__()
        self.model_cfg = model_cfg
        self.embed = OneHotEmbed(model_cfg)
        self.attn = Attention(model_cfg)
        self.unembed = OneHotUnembed(model_cfg)

        self.activation_cache = {}


    def forward(self, tokens):
        # tokens [batch, pos]
        # output [batch, pos, d_vocab]

        input = tokens.to(device)
        resid_pre = self.embed(input)
        resid_post = resid_pre + self.attn(resid_pre)
        out = self.unembed(resid_post)

        # add activations to the same cache as before
        self.activation_cache = self.attn.attn_activation_cache
        self.activation_cache['hook_resid_pre'] = resid_pre
        self.activation_cache['hook_resid_post'] = resid_post
        self.activation_cache['unembed'] = out

        return out
    

attn_only_1l = OneLayerAttnOnly(model_cfg).to(device)
attn_only_1l

        

OneLayerAttnOnly(
  (embed): OneHotEmbed(
    (embedding): Embedding(12, 12)
  )
  (attn): Attention()
  (unembed): OneHotUnembed()
)

In [9]:
def plot_metrics(train_loss,train_acc, test_loss, test_acc):
    fig = make_subplots(rows=2,cols=2, subplot_titles = ['Train Loss', 'Train Accuracy', 'Test Loss', 'Test Accuracy'])
    fig.add_trace(go.Scatter(y=train_loss), row=1, col=1)
    fig.add_trace(go.Scatter(y=train_acc), row=1, col=2)
    fig.add_trace(go.Scatter(y=test_loss), row=2, col=1)
    fig.add_trace(go.Scatter(y=test_acc), row=2, col=2)
    # get rid of legend
    fig.update_layout(showlegend=False)
    ## add x and y axis labels and change plot style 
    fig.update_xaxes(
        title = 'Epoch',
        zeroline=True,
        zerolinewidth=1,
        zerolinecolor='black',
        showgrid=False
    )
    fig.update_yaxes(
        title = 'Loss',
        showgrid=False,
        col=1
    )
    fig.update_yaxes(
        title = 'Accuracy',
        showgrid=False,
        col=2
    )
    fig.update_layout(
        font = dict(size =12),
        width=800,
        height =500,
        margin=dict(l=10, 
                    r = 20,
                    b=10,
                    t=20,
                    pad=0)
    )
    fig.show()

In [10]:
@dataclass
class train_cfg:
    ctx_length: int = 4
    trainset_size: int = 100000
    valset_size: int = 1000
    prob: int = 2
    epochs: int = 15
    batch_size: int = 512
    weight_decay: float = 0.0
    seed: int = 42
    lr: float = 1e-3

class Trainer:

    def __init__(self, train_cfg, model_cfg):
        self.train_cfg = train_cfg
        self.model = OneLayerAttnOnly(model_cfg).to(device)


    def train_dataloader(self, seed: int):
        trainset = SkipTrigramDataset(size=self.train_cfg.trainset_size,
                                      prob=self.train_cfg.prob,
                                      prompt_length=self.train_cfg.ctx_length)

        return DataLoader(trainset, batch_size=self.train_cfg.batch_size, shuffle=True)

    def val_dataloader(self, seed: int):
        valset = SkipTrigramDataset(size=self.train_cfg.valset_size,
                                    prob = self.train_cfg.prob,
                                    prompt_length=self.train_cfg.ctx_length)
        return DataLoader(valset, batch_size=self.train_cfg.batch_size, shuffle=False)

    def configure_optimizers(self):
        optimizer = t.optim.Adam(self.model.parameters(), lr=self.train_cfg.lr, weight_decay=self.train_cfg.weight_decay)
        return optimizer

In [11]:



def train(model, train_data, criterion, optimizer):

    avg_loss = 0
    acc = 0

    for _, batch in list(enumerate(train_data)):

        X = batch[0].to(device)
        y = batch[1].squeeze().to(device)
        assert X.shape[0] == y.shape[0]

        output = model(X)[:,-1]

        with t.no_grad():
            corr = (output.argmax(dim=-1) == y).count_nonzero()
        loss = criterion(output,y)
        loss.backward()

        optimizer.step()
        avg_loss += loss.item() * len(batch)
        acc += corr.item()
        optimizer.zero_grad()

    cache = model

    return avg_loss/train_cfg.trainset_size, acc/train_cfg.trainset_size, cache
    
def test(model, test_data, criterion):

    avg_loss = 0
    acc = 0

    for _, batch in list(enumerate(test_data)):
        X = batch[0].to(device)
        y = batch[1].squeeze().to(device)
        assert X.shape[0] == y.shape[0]

        # don't log the gradients for testing
        with t.no_grad():
            output = model(X)[:,-1]
        loss = criterion(output,y)
        corr = (output.argmax(dim=-1) == y).count_nonzero()
        avg_loss += loss.item() * len(batch)
        acc += corr.item()


    return avg_loss/train_cfg.valset_size, acc/train_cfg.valset_size

In [12]:
from tqdm import tqdm 

def train_and_eval(train_cfg, model_cfg, save_caches = []):

    trainer = Trainer(train_cfg, model_cfg)
    model = trainer.model
    train_data = trainer.train_dataloader(seed=train_cfg.seed)

    test_data = trainer.val_dataloader(seed=train_cfg.seed)
    criterion = nn.CrossEntropyLoss()
    optimizer = trainer.configure_optimizers()
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []

    for n in tqdm(range(train_cfg.epochs)):
        loss, acc, cache = train(model, train_data, criterion, optimizer)

        train_loss.append(loss)
        train_acc.append(acc)
        t_loss, t_acc = test(model, test_data, criterion)
        test_loss.append(t_loss)
        test_acc.append(t_acc)
        print(f"train loss = {loss}, train acc = {acc}, test loss = {t_loss}, test acc = {t_acc}")

        if n in save_caches:
            t.save(cache, f'saved_models/epoch_{n}_dhead_{model_cfg.d_head}_nheads_{model_cfg.n_heads}.pth')
            print(f"checkpoint saved to saved_models/epoch_{n}_dhead_{model_cfg.d_head}_nheads_{model_cfg.n_heads}, shape test = {cache.activation_cache['hook_q'].shape} ")

    return train_loss, train_acc, test_loss, test_acc, trainer.model




Helper functions for plotting

In [13]:
from transformer_lens import utils, HookedTransformer, ActivationCache

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [14]:
def get_head_to_resid(model, cache, head, seq):
  model(seq)
  z = cache["hook_z"][:, :, head, :]
  wo = model.attn.W_O[head, :, :]
  bo = model.attn.b_O
  out = einsum("batch pos d_head, d_head d_model -> batch pos d_model", z, wo) + bo
  return out

def get_head_to_logits(model, head_to_resid):
  return model.unembed(head_to_resid)

def show_pattern(cache, seq, model):
    X = t.Tensor(seq).unsqueeze(0).int()
    model(X)
    ax_label = [f"{i}: {z.item()}" for i, z in enumerate(X[0])]
    imshow(cache['hook_pattern'][0], facet_col = 0, x = ax_label, y = ax_label)

def get_head_to_resid(model, cache, head, seq):
  model(seq)
  z = cache["hook_z"][:, :, head, :]
  wo = model.attn.W_O[head, :, :]
  bo = model.attn.b_O
  out = einsum("batch pos d_head, d_head d_model -> batch pos d_model", z, wo) + bo
  return out

def get_head_to_logits(model, head_to_resid):
  return model.unembed(head_to_resid)





Running for many iterations of d_head

In [15]:
train_cfg.epochs
[i for i in range(10,train_cfg.epochs,10)]
list(range(1,train_cfg.epochs,10))

[1, 11]

In [16]:
train_losses = {}
train_accs = {}
test_losses = {}
test_accs = {}
models = {}
caches = {}
dh_list = [1,2,3,5,10]
for dh in dh_list:

    train_config = train_cfg(
        ctx_length=4,
        weight_decay=0,
        trainset_size = 100000,
        prob = 2,
        epochs = 20,
        batch_size = 512
    )

    model_config = model_cfg(
        d_model = 11,
        d_vocab = 11,
        d_head = dh,
        init_range = 0.02,
        n_ctx = 4,
        n_heads = 2,
        n_layers = 1)

    train_losses[dh], train_accs[dh], test_losses[dh], test_accs[dh], models[dh] = train_and_eval(train_config, model_config, save_caches = list(range(12)))
    #train_losses[dh], train_accs[dh], test_losses[dh], test_accs[dh], models[dh] = train_and_eval(train_config, model_config)

  5%|▌         | 1/20 [00:00<00:08,  2.27it/s]

train loss = 0.009550961723327638, train acc = 0.07676, test loss = 0.009638270854949952, test acc = 0.084
checkpoint saved to saved_models/epoch_0_dhead_1_nheads_2, shape test = torch.Size([488, 4, 2, 1]) 


 10%|█         | 2/20 [00:00<00:07,  2.37it/s]

train loss = 0.009302466802597047, train acc = 0.10172, test loss = 0.009314502716064453, test acc = 0.168
checkpoint saved to saved_models/epoch_1_dhead_1_nheads_2, shape test = torch.Size([488, 4, 2, 1]) 


 15%|█▌        | 3/20 [00:01<00:07,  2.32it/s]

...

train loss = 0.0047139251899719235, train acc = 0.54505, test loss = 0.0048037576675415035, test acc = 0.546


 95%|█████████▌| 19/20 [00:11<00:00,  1.73it/s]

train loss = 0.004714181468486786, train acc = 0.54543, test loss = 0.004803736925125122, test acc = 0.542


100%|██████████| 20/20 [00:11<00:00,  1.70it/s]

train loss = 0.0047139433693885804, train acc = 0.54551, test loss = 0.004806351661682129, test acc = 0.544


In [74]:
ans_map

{(1, 1): 0,
 (1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (2, 2): 4,
 (2, 3): 5,
 (2, 4): 6,
 (3, 3): 7,
 (3, 4): 8,
 (4, 4): 9}

In [17]:
def get_rankings(seq, model):
    keys = seq[:-1]
    X = t.Tensor(seq).unsqueeze(0).int()
    logits = model(X)
    scores = model.activation_cache['hook_attn_scores'][0]
    rankings = {}

    for head in range(model_config.n_heads):
        ## get scores of attention from last query pos to all other (key) positions
        rankings[head] = sorted(list(zip(range(len(keys)),keys, scores[head,-1,:-1].detach())), key = lambda x: x[-1], reverse = True)

    return rankings
    

In [91]:
model = checkpoints[5][1]

keys = seq[:-1]
X = t.Tensor(seq).unsqueeze(0).int()
logits = model(X)
scores = model.activation_cache['hook_attn_scores'][0]

head = 0 

scores[head, -1, :-1].detach()

tensor([-1.7322,  5.6003,  4.0497,  4.4429])

In [32]:
print(train_config.epochs, train_config.batch_size, train_config.trainset_size)
512/8, 488/8

60 512 100000


(64.0, 61.0)

In [20]:
checkpoints = {}
for dh in dh_list:
    ## model learns hierarchy very quickly 
    #checkpoints[dh] = [t.load(f'saved_models/epoch_{n}_dhead_{dh}_nheads_{model_config.n_heads}.pth') for n in [1,11,21,31,41,51]]

    checkpoints[dh] = [t.load(f'saved_models/epoch_{n}_dhead_{dh}_nheads_{model_config.n_heads}.pth') for n in range(12)]

#checkpoints[5][1].activation_cache['hook_q'][1],models[5].activation_cache['hook_q'][1]

check to see which models are 'hierarchical' and which are 'split attention'

In [18]:
def is_hier(model, seq):
    return t.all(t.flip(t.tensor(get_rankings(seq, model)[0]).T[1], dims = (0,)) == t.tensor(get_rankings(seq, model)[1]).T[1])

d_head_H = []
d_head_S = []
seq = [1,2,3,4,0]
for dh in dh_list:
    if is_hier(models[dh], seq):
        d_head_H.append(dh)
    else:
        d_head_S.append(dh)
        print(get_rankings(seq, models[dh]))

d_head_H, d_head_S

{0: [(2, 3, tensor(12.6628)), (0, 1, tensor(5.9790)), (1, 2, tensor(-0.3850)), (3, 4, tensor(-1.7360))], 1: [(3, 4, tensor(7.3367)), (0, 1, tensor(6.7015)), (2, 3, tensor(2.9687)), (1, 2, tensor(-0.6698))]}


([2, 3, 5, 10], [1])

In [92]:
def get_scores(model, seq):
    X = t.Tensor(seq).unsqueeze(0).int()
    logits = model(X)
    scores = model.activation_cache['hook_attn_scores'][0]
    return scores

In [103]:
def get_attn_evol(dh):
    cached_models = checkpoints[dh]+[models[dh]]
    y_labels = [str(i) for i in range(len(checkpoints[dh]))] + [str(train_config.epochs)]
    x_labels = [str(i) for i in range(1,5)]
    seq = [1,2,3,4,0]


    attn_list = t.stack([t.stack([get_scores(cached_models[k], seq)[head, -1, :-1].detach() for k in range(len(cached_models))]) for head in range(model_config.n_heads)])

    imshow(attn_list, facet_col = 0, y = y_labels, x = x_labels, title = f"Attention Scores per Source Token across epochs for d_head = {dh}",xaxis = "Source Token", yaxis = "Epoch")
for dh in dh_list:
    get_attn_evol(dh)



In [22]:
indx = list(ans_map.keys())
indx.append("nan")
x_labels = [str(x) for x in indx]

for dh in [10]:
    cached_models = checkpoints[dh]+[models[dh]]
    for k, pre_trained in enumerate(cached_models):
        head_outputs = t.zeros((model_config.n_heads, model_config.n_ctx, model_config.d_vocab))

        ## getting DLA from each token to each possible continuation
        for i in range(1,5):
            seq = [i,0]
            X = t.Tensor(seq).unsqueeze(0).int()
            for head in range(model_config.n_heads):
                temp = get_head_to_resid(pre_trained, pre_trained.activation_cache, head, X)
                temp = get_head_to_logits(pre_trained, temp)
                #print(head_outputs[head, i-1].shape, temp[0,-1].shape)
                head_outputs[head, i-1,:] = temp[0,-1,:]
        
        seq = [1,2,3,4,0]
        rankings = get_rankings(seq, pre_trained)


        ## getting attention of each token per head


        print(f"Model with d_head = {dh} and num_model = {k} is hierarchical? {is_hier(pre_trained, seq)}")
        print(f"Attention ranking is {rankings}")
        imshow(head_outputs, facet_col = 0, y = [1,2,3,4], x = x_labels, title = f"Head Outputs Conditional on Attn for d_head = {dh}",xaxis = "Logit contribution to completions", yaxis = "Source token")

Model with d_head = 10 and num_model = 0 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(2.8099)), (1, 2, tensor(2.4868)), (2, 3, tensor(1.9177)), (3, 4, tensor(1.0974))], 1: [(3, 4, tensor(2.7342)), (2, 3, tensor(2.6214)), (1, 2, tensor(1.7642)), (0, 1, tensor(1.6657))]}


Model with d_head = 10 and num_model = 1 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(9.8990)), (1, 2, tensor(6.9153)), (2, 3, tensor(0.3471)), (3, 4, tensor(-2.6852))], 1: [(3, 4, tensor(9.8829)), (2, 3, tensor(8.5953)), (1, 2, tensor(4.3720)), (0, 1, tensor(-2.1661))]}


Model with d_head = 10 and num_model = 2 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(13.1496)), (1, 2, tensor(8.6392)), (2, 3, tensor(2.4490)), (3, 4, tensor(-4.9239))], 1: [(3, 4, tensor(13.3494)), (2, 3, tensor(10.0934)), (1, 2, tensor(4.3774)), (0, 1, tensor(-3.5217))]}


Model with d_head = 10 and num_model = 3 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(14.9161)), (1, 2, tensor(9.8456)), (2, 3, tensor(3.1012)), (3, 4, tensor(-5.3660))], 1: [(3, 4, tensor(15.2885)), (2, 3, tensor(10.5594)), (1, 2, tensor(4.0707)), (0, 1, tensor(-4.1092))]}


Model with d_head = 10 and num_model = 4 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(15.9688)), (1, 2, tensor(10.4439)), (2, 3, tensor(3.3155)), (3, 4, tensor(-5.5505))], 1: [(3, 4, tensor(16.4898)), (2, 3, tensor(10.9229)), (1, 2, tensor(3.9219)), (0, 1, tensor(-4.5394))]}


...

Model with d_head = 10 and num_model = 8 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(17.9871)), (1, 2, tensor(11.5830)), (2, 3, tensor(3.4143)), (3, 4, tensor(-5.9302))], 1: [(3, 4, tensor(18.8846)), (2, 3, tensor(11.8284)), (1, 2, tensor(3.7506)), (0, 1, tensor(-5.3670))]}


Model with d_head = 10 and num_model = 9 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(18.2992)), (1, 2, tensor(11.7464)), (2, 3, tensor(3.4454)), (3, 4, tensor(-5.9888))], 1: [(3, 4, tensor(19.2438)), (2, 3, tensor(11.9858)), (1, 2, tensor(3.7400)), (0, 1, tensor(-5.4715))]}


Model with d_head = 10 and num_model = 10 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(18.5647)), (1, 2, tensor(11.8805)), (2, 3, tensor(3.4564)), (3, 4, tensor(-6.0294))], 1: [(3, 4, tensor(19.5608)), (2, 3, tensor(12.1286)), (1, 2, tensor(3.7634)), (0, 1, tensor(-5.5669))]}


Model with d_head = 10 and num_model = 11 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(18.8036)), (1, 2, tensor(12.0083)), (2, 3, tensor(3.4670)), (3, 4, tensor(-6.0628))], 1: [(3, 4, tensor(19.8423)), (2, 3, tensor(12.2524)), (1, 2, tensor(3.7613)), (0, 1, tensor(-5.6389))]}


Model with d_head = 10 and num_model = 12 is hierarchical? True
Attention ranking is {0: [(0, 1, tensor(20.2136)), (1, 2, tensor(12.8087)), (2, 3, tensor(3.5100)), (3, 4, tensor(-6.4260))], 1: [(3, 4, tensor(21.4626)), (2, 3, tensor(12.9959)), (1, 2, tensor(3.8334)), (0, 1, tensor(-6.0711))]}


In [88]:
## save models where heads share attention to the same source token as a different way of acheiving the task

#t.save(models[1], 'dhead1_new.pth')


In [96]:
ans_map

{(1, 1): 0,
 (1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (2, 2): 4,
 (2, 3): 5,
 (2, 4): 6,
 (3, 3): 7,
 (3, 4): 8,
 (4, 4): 9}

For binomial distribution (k=1 Bernoulli trials), standard error for a sample stat is \sqrt(pq/n) ~ .015 for the val set, so possibly overfitting is pulling us out of 

In [116]:
test_err = np.sqrt(1/(4*train_config.valset_size))

train_err = np.sqrt(1/(4*train_config.trainset_size))
for ind in dh_list:
    #print(train_losses[ind][-1],train_accs[ind][-1], test_losses[ind][-1], test_accs[ind][-1])
    print(f"For d_head = {ind}, the final test accuracy is {test_accs[ind][-1]}, the max test accuracy is {np.max(test_accs[ind])} at epoch {np.argmax(test_accs[ind])}/{train_config.epochs}")
    print(f"For d_head = {ind}, the final training accuracy is {train_accs[ind][-1]}, the max test accuracy is {np.max(train_accs[ind])} at epoch {np.argmax(train_accs[ind])}/{train_config.epochs}")

    print(f"test error within range? {np.abs(test_accs[ind][-1] - np.max(test_accs[ind])) <= test_err}")

    print(f"train error within range? {np.abs(train_accs[ind][-1] - np.max(train_accs[ind])) <= test_err}")
    plot_metrics(train_losses[ind],train_accs[ind], test_losses[ind], test_accs[ind]) 

For d_head = 1, the final test accuracy is 0.52, the max test accuracy is 0.52 at epoch 16/20
For d_head = 1, the final training accuracy is 0.51348, the max test accuracy is 0.51348 at epoch 17/20
test error within range? True
train error within range? True


For d_head = 2, the final test accuracy is 0.547, the max test accuracy is 0.547 at epoch 4/20
For d_head = 2, the final training accuracy is 0.54471, the max test accuracy is 0.54493 at epoch 18/20
test error within range? True
train error within range? True


For d_head = 3, the final test accuracy is 0.545, the max test accuracy is 0.55 at epoch 10/20
For d_head = 3, the final training accuracy is 0.54444, the max test accuracy is 0.54486 at epoch 9/20
test error within range? True
train error within range? True


For d_head = 5, the final test accuracy is 0.55, the max test accuracy is 0.553 at epoch 14/20
For d_head = 5, the final training accuracy is 0.54499, the max test accuracy is 0.54541 at epoch 18/20
test error within range? True
train error within range? True


For d_head = 10, the final test accuracy is 0.544, the max test accuracy is 0.556 at epoch 14/20
For d_head = 10, the final training accuracy is 0.54551, the max test accuracy is 0.54582 at epoch 16/20
test error within range? True
train error within range? True


In [91]:
test_signal = SkipTrigramDataset(size = 10000,
                                     prob = 1,
                                     prompt_length = train_config.ctx_length)

for dh in [1,2,3,4,5]:
    model = models[dh]
    acc = 0

    for i in range(len(test_signal)):
        X = test_signal[i][0].unsqueeze(0) # add batch dim to feed into model
        y = test_signal[i][1].item()

        output = model(X)[:,-1]
        corr = (output.argmax(dim=-1) == y).count_nonzero()
        acc += corr.item()

    print(acc/len(test_signal))




1.0
1.0
1.0
1.0
1.0


In [92]:
seq = [3,5,2,0]
for i in range(1,6):
    print(get_rankings(seq, models[i]))
    show_pattern(models[i].activation_cache, seq, models[i])

{0: [(0, 3, tensor(2.6849)), (2, 2, tensor(2.4218)), (1, 5, tensor(-7.2294))], 1: [(0, 3, tensor(11.4649)), (2, 2, tensor(3.0279)), (1, 5, tensor(-3.0561))]}


{0: [(2, 2, tensor(5.5252)), (0, 3, tensor(-1.1040)), (1, 5, tensor(-6.0368))], 1: [(0, 3, tensor(11.0803)), (2, 2, tensor(10.9012)), (1, 5, tensor(-7.2460))]}


{0: [(2, 2, tensor(14.4581)), (0, 3, tensor(-2.5995)), (1, 5, tensor(-10.5149))], 1: [(0, 3, tensor(11.7716)), (2, 2, tensor(8.4585)), (1, 5, tensor(-12.0351))]}


{0: [(2, 2, tensor(22.8451)), (0, 3, tensor(4.5868)), (1, 5, tensor(-14.3393))], 1: [(0, 3, tensor(16.7878)), (2, 2, tensor(-2.9431)), (1, 5, tensor(-20.8075))]}


{0: [(2, 2, tensor(20.5462)), (0, 3, tensor(-3.5068)), (1, 5, tensor(-15.0189))], 1: [(0, 3, tensor(22.3207)), (2, 2, tensor(-7.4202)), (1, 5, tensor(-15.3385))]}


Does the QK matrix store things orthogonally? (It's not sparse?)

In [127]:
def cos_sim(v1, v2):
    with t.no_grad():
        out = einsum("d, d -> ", v1, v2)/(t.norm(v1) * t.norm(v2))
    return out

assert cos_sim(t.tensor([1.,0.,0.]), t.tensor([0.,4.,0.])) == t.tensor(0.)
assert cos_sim(t.tensor([1.,0.,0.]), t.tensor([2.,0.,0.])) == t.tensor(1.)

In [152]:
## Queries have shape [batch, pos_q, n_heads, d_head]
## Keys have shape [batch, pos_k, n_heads, d_head]
model = models[5]

queries = rearrange(model.activation_cache['hook_q'][0], "q nh dh -> nh q dh") #remove batch dim and rearrange so that d_head is first
keys = rearrange(model.activation_cache['hook_k'][0], "q nh dh -> nh q dh") #remove batch dim and rearrange so that d_head is first

## get the cosim for all pairs of keys and queries

head_idx = 0
[[cos_sim(keys[head_idx,i], queries[head_idx, j]) for j in range(queries.shape[1])] for i in range(keys.shape[1])] ## [pos_k, pos_q]


[[tensor(0.9921), tensor(0.8856), tensor(0.9923), tensor(0.9983)],
 [tensor(-0.8149), tensor(-0.4114), tensor(-0.8022), tensor(-0.7912)],
 [tensor(0.9962), tensor(0.8557), tensor(0.9952), tensor(0.9989)],
 [tensor(-0.9923), tensor(-0.8798), tensor(-0.9922), tensor(-0.9984)]]